In [ ]:
import os
from pprint import pprint
from dotenv import load_dotenv

# LangChain 관련 모듈 import
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# .env환경 변수
load_dotenv()

# --- 0단계: 문서 로드 ---
print(">>> 0단계: 문서를 로드합니다...")
# PyPDFLoader를 사용하여 PDF 문서를 로드합니다.
# 사용자의 디렉토리 구조에 맞춰 상대 경로를 설정했습니다.
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"문서의 총 페이지 수: {len(documents)}")


# --- 1단계: 문서 분할 ---
print("\n>>> 1단계: 문서를 분할합니다...")
# 법률 문서의 구조적 특징을 고려한 구분자(separators)를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
split_docs = text_splitter.split_documents(documents)
print(f"분할된 문서(청크)의 수: {len(split_docs)}")


# --- 2단계: 임베딩 모델 설정 ---
print("\n>>> 2단계: 임베딩 모델을 설정합니다...")
# Upstage의 solar-embedding-1-large 모델을 사용하여 텍스트를 벡터로 변환합니다.
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")


# --- 3단계: 벡터 저장소(Vector Store) 및 검색기(Retriever) 설정 ---
print("\n>>> 3단계: 벡터 저장소를 만들고 검색기를 설정합니다...")
# 분할된 문서를 임베딩하여 FAISS 벡터 저장소에 저장합니다.
vectorstore = FAISS.from_documents(split_docs, embeddings)
# 벡터 저장소를 기반으로 검색기를 생성합니다. 상위 5개의 관련 문서를 검색합니다.
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


# --- 4단계: LLM 설정 ---
print("\n>>> 4단계: LLM을 설정합니다...")
# Upstage의 Solar 모델을 LLM으로 사용합니다.
llm = ChatUpstage(
    model_name="solar-1-mini-chat", # solar-1-mini-chat 사용 (solar-pro는 현재 지원되지 않음)
    temperature=0.2
)


# --- 5단계: 법률 자문 프롬프트 작성 ---
print("\n>>> 5단계: 프롬프트를 작성합니다...")
prompt_template_str = """
당신은 콘텐츠 분야 전문 법률 자문가입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate.from_template(prompt_template_str)


# --- 6단계: QA 체인 생성 ---
print("\n>>> 6단계: QA 체인을 생성합니다...")
# RetrievalQA 체인을 생성하여 RAG 파이프라인을 완성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


# --- 7단계: 테스트 질문 작성 ---
print("\n>>> 7단계: 테스트 질문을 준비합니다...")
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


# --- 8단계: 분쟁 유형 분류 함수 (선택) ---
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# --- 테스트 실행 ---
print("\n" + "="*50)
print(" RAG 법률 자문 시스템 테스트 시작")
print("="*50 + "\n")

for i, question in enumerate(test_questions):
    dispute_type = classify_dispute_type(question)
    
    print(f"--- [질문 {i+1}] ---")
    print(f"유형: {dispute_type}")
    print(f"내용: {question}")
    
    # QA 체인 실행
    result = qa_chain.invoke(question)
    
    print("\n[답변]")
    pprint(result["result"])
    
    # print("\n[참조한 원문]")
    # for doc in result["source_documents"]:
    #     pprint(doc.page_content)
    #     print("-" * 20)
        
    print("\n" + "="*50 + "\n")

>>> 0단계: 문서를 로드합니다...
문서의 총 페이지 수: 109

>>> 1단계: 문서를 분할합니다...
분할된 문서(청크)의 수: 104

>>> 2단계: 임베딩 모델을 설정합니다...

>>> 3단계: 벡터 저장소를 만들고 검색기를 설정합니다...

>>> 4단계: LLM을 설정합니다...

>>> 5단계: 프롬프트를 작성합니다...

>>> 6단계: QA 체인을 생성합니다...

>>> 7단계: 테스트 질문을 준비합니다...

 RAG 법률 자문 시스템 테스트 시작

--- [질문 1] ---
유형: 게임
내용: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[답변]
('제시된 사례들을 근거로 답변드리겠습니다.\n'
 '\n'
 '1. 관련 법령 및 조항:\n'
 '   - 민법 제250조(도품, 유실물에 대한 특례)\n'
 '   - 민법 제251조(양수인의 선의취득)\n'
 '   - 게임회사의 이용약관(특히, 이용자의 의무 및 아이템 복구 관련 규정)\n'
 '\n'
 '2. 비슷한 사례의 처리경위와 결과:\n'
 '   - 사례 1(2006년): 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 거부되었습니다.\n'
 '   - 사례 2(2009년): 시스템 오류로 게임머니가 소실되었다고 주장했으나, 시스템 오류가 아닌 통신회선 또는 PC 오류로 판단되어 '
 '복구가 거부되었습니다.\n'
 '   - 사례 4(2006년): 프로그램 오류로 아이템이 소멸되었으나, 게임회사가 오류를 인정하고 복구를 해주었습니다.\n'
 '\n'
 '3. 실무적 해결방안:\n'
 '   - 단계 1: 게임회사에 문의하여 시스템 오류 발생 시점, 오류 내용, 소멸된 아이템 정보 등을 자세히 기록하고 문의합니다.\n'
 '   - 단계 2: 게임회사의 이용약관을 확인하고, 아이템 복구 관련 규정을 숙지합니다.\n'
 '   - 단계 3: 게임회사의 고객센터를 통해 공식적으로 아